# Dimensionality Reduction using GPU Accelerated PCA in RAPIDS
#### Ivana Jovicic
-------

While the world’s data doubles each year, CPU computing has hit a brick wall with the end of Moore’s law. For the same reasons, scientific computing and deep learning has turned to NVIDIA GPU acceleration, data analytics and machine learning where GPU acceleration is ideal. 

NVIDIA created RAPIDS – an open-source data analytics and machine learning acceleration platform that leverages GPUs to accelerate computations. RAPIDS is based on Python, has pandas-like and Scikit-Learn-like interfaces, is built on Apache Arrow in-memory data format, and can scale from 1 to multi-GPU to multi-nodes. RAPIDS integrates easily into the world’s most popular data science Python-based workflows. RAPIDS accelerates data science end-to-end – from data prep, to machine learning, to deep learning. And through Arrow, Spark users can easily move data into the RAPIDS platform for acceleration.

In this notebook, we will also show how to use PCA - a popular dimensionality reduction algorithm - and how to use the GPU accelerated implementation of this algorithm in RAPIDS.

**Table of Contents**

* Clustering with PCA
* Why do we use PCA?
* Setup
* Basic Example using Iris Dataset
  * Importing Libraries
  * Import Iris Dataset
  * Dataset Overview
  * Basic Cleaning of the Data
  * Applying PCA
  * Training and Making Predictions
* PCA on Larger Dataset and CPU vs GPU comparisson
  * Benchmark with Random Data
* Conclusion

Before going any further, let's make sure we have access to `matplotlib`, a popular Python library for data visualization.

In [ ]:
import os

try:
    import matplotlib
except ModuleNotFoundError:
    os.system('conda install -y matplotlib')

## Dimensionality Reductino with PCA

*Principal component analysis (PCA)* is a technique used for identification of a smaller number of uncorrelated variables known as principal components from a larger set of data. The technique is widely used to emphasize variation and capture strong patterns in a data set. 

Principal component analysis is considered a useful statistical method and used in fields such as image compression, face recognition, neuroscience and computer graphics.

source : https://www.techopedia.com/definition/32509/principal-component-analysis-pca

## Why do we use PCA ?

Large number of features in the dataset is one of the factors that affect both the training time as well as accuracy of machine learning models. You have different **options to deal with huge number of features in a dataset**.

1. Try to train the models on original number of features, which take days or weeks if the number of features is too high.
2. Reduce the number of variables by merging correlated variables.
3. Extract the most important features from the dataset that are responsible for maximum variance in the output. Different statistical techniques are used for this purpose e.g. linear discriminant analysis, factor analysis, and principal component analysis.

Principal component analysis, or PCA, is a statistical technique to **convert high dimensional data to low dimensional data** by selecting the most important features that capture maximum information about the dataset. 

The features are selected on the basis of variance that they cause in the output. The feature that causes highest variance is the *first principal component*. The feature that is responsible for second highest variance is considered the *second principal component*, and so on. It is important to mention that principal components do not have any correlation with each other.

source: https://stackabuse.com/implementing-pca-in-python-with-scikit-learn/

## Setup

This notebook was tested using the `nvcr.io/nvidia/rapidsai/rapidsai:0.5-cuda10.0-runtime-ubuntu18.04-gcc7-py3.7` Docker container from [NVIDIA GPU Cloud](https://ngc.nvidia.com) and run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Let's check out our hardware setup by runing the `nvidia-smi` command:

In [ ]:
!nvidia-smi

Next, let's see what CUDA version we have.

In [ ]:
!nvcc --version

## Basic Example using Iris Dataset

### Importing  libraries

At first, we are going to import basic python libraries needed to run a very simple exaple to explain the above mentioned algorithm

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as skPCA


import os

### Import Iris Dataset

https://archive.ics.uci.edu/ml/datasets/iris

The dataset consists of 150 records of Iris plant with **four features**: 'sepal-length', 'sepal-width', 'petal-length', and 'petal-width'. All of the features are numeric. The records have been classified into one of the **three classes** i.e. 'Iris-setosa', 'Iris-versicolor', or 'Iris-verginica'.

### Dataset Overview

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
#source : https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

# The indices of the features that we are plotting
x_index = 0
y_index = 1

# this formatter will label the colorbar with the correct target names
formatter = plt.FuncFormatter(lambda i, *args: iris.target_names[int(i)])

plt.figure(figsize=(5, 4))
plt.scatter(iris.data[:, x_index], iris.data[:, y_index], c=iris.target)
plt.colorbar(ticks=[0, 1, 2], format=formatter)
plt.xlabel(iris.feature_names[x_index])
plt.ylabel(iris.feature_names[y_index])



plt.tight_layout()
plt.show()

In [ ]:
print(iris.data.shape) ## number of rows/columns
print(iris.feature_names) ## feature names
print(iris.target_names) ## categories 

### Basic Cleaning of the Data

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"  
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']  
df = pd.read_csv(url, names=names)  
df.head()

In [ ]:
#divide the dataset into a feature set and corresponding labels.
X = df.drop('Class', 1)  
y = df['Class']  

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  


In [ ]:
#PCA performs best with a normalized feature set. We will perform standard scalar normalization to normalize our feature set.
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

### Applying PCA

Performing PCA using Scikit-Learn is a two-step process:

1. **Initialize the PCA class** by passing the number of components to the constructor.
2. **Call the *fit* and then *transform* methods** by passing the feature set to these methods. The *transform* method returns the specified number of principal components.

In [ ]:
pca = skPCA()  
X_train = pca.fit_transform(X_train)  
X_test = pca.transform(X_test)  

The PCA class contains `explained_variance_ratio_` which returns the variance caused by each of the principal components.

In [ ]:
explained_variance = pca.explained_variance_ratio_  
print(explained_variance)

It can be seen that first principal component is responsible for 72.22% variance. Similarly, the second principal component causes 23.9% variance in the dataset. Collectively we can say that (72.22 + 23.9) 96.21% percent of the classification information contained in the feature set is captured by the first two principal components.

In [ ]:
pca = skPCA(n_components=1)  
X_train = pca.fit_transform(X_train)  
X_test = pca.transform(X_test) 

### Training and Making Predictions
In this case we'll use random forest classification for making the predictions.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)  
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
#Performance Evaluation
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)  
print(cm)  
print( accuracy_score(y_test, y_pred))  

it can be seen from the output that with only one feature, the random forest algorithm is able to correctly predict 28 out of 30 instances, resulting in 93.33% accuracy.

## PCA on Larger Dataset and CPU vs GPU comparisson  

We will first import some helper functions

In [ ]:
# measure the execution time 
from timeit import default_timer


class Timer(object):  #Class for timing execution speed of small code snippets.
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [ ]:
def load_data(nrows, ncols, ):
    print('use random data')
    X = np.random.rand(nrows,ncols)
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    return df

In [ ]:
from sklearn.metrics import mean_squared_error


def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x    

### Benchmark with Random Data


Let's use the larger dataset with more features

In [ ]:
%%time
## here, we will load the random data with 1M rows and 400 columns. Feel free to experiment with a different dataset sizes. 
nrows = 1000000
ncols = 400

X = load_data(nrows,ncols)
print('data',X.shape)

Let's take a look into all possible parameters that we can use when applying PCA :http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

We will start here with the following : 

- **n_components : int, float, None or string** <br>
Number of components to keep. if `n_components` is not set all components are kept
- **whiten : bool, optional (default False)** <br>
When `True` (`False` by default) the `components_` vectors are multiplied by the square root of `n_samples` and then divided by the singular values to ensure uncorrelated outputs with unit component-wise variances.<br/>Whitening will remove some information from the transformed signal (the relative variance scales of the components) but can sometime improve the predictive accuracy of the downstream estimators by making their data respect some hard-wired assumptions

- **random_state : int, RandomState instance or None, optional (default None)** <br>
If int, `random_state` is the seed used by the random number generator

- **svd_solver : string {‘auto’, ‘full’, ‘arpack’, ‘randomized’}** <br>
If `"full"` :run exact full SVD calling the standard LAPACK solver via scipy.linalg.svd and select the components by postprocessing

In [ ]:
n_components = 10
whiten = False
random_state = 42
svd_solver="full"

Let's check the time needed to execute PCA  function using standard *sklearn* library. Note: this algorithm runs on CPU only. 

In [ ]:
import multiprocessing
print(multiprocessing.cpu_count()) # Return the number of CPUs in the system. 

In [ ]:
%%time
from sklearn.decomposition import PCA as skPCA 

pca_sk = skPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_sk = pca_sk.fit_transform(X)

Now, before we execute PCA function using RAPIDS *cuml* library we will first read the data in  GPU data format using *cudf* :
- cudf - GPU DataFrame manipulation library https://github.com/rapidsai/cudf
- cuml - suite of libraries that implements a machine learning algorithms within the RAPIDS data science ecosystem https://github.com/rapidsai/cuml

In [ ]:
%%time
import cudf
import cuml #documentaton can be found here https://cuml.readthedocs.io/en/latest/
from cuml import PCA as cumlPCA  


X = cudf.DataFrame.from_pandas(X)

Next, we will execute the PCA function using cuml and check the performance

In [ ]:
%%time
pca_cuml = cumlPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_cuml = pca_cuml.fit_transform(X)

We see that for dataset of size 1000000 rows and 400 columns it takes around 8X less time to execute the PCA algorithm using RAPIDS cuml library.

Block of the code below compares the attributes and results of two libraries : *sklearn* PCA and *cuml* PCA. Here, we see that results and attributes are exacty the same and that user will not see any difference in the existing workflow. 

In [ ]:
for attr in ['singular_values_','components_','explained_variance_',
             'explained_variance_ratio_']:
    passed = array_equal(getattr(pca_sk,attr),getattr(pca_cuml,attr))
    message = 'compare pca: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

In [ ]:
passed = array_equal(result_sk,result_cuml)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

## Conclusion

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)
